In [0]:
import numpy as np

def reflectSuppress(Im,h,epsilon):
    Y = Im     
    m, n, r = Y.shape
    T = np.zeros((m,n,r))
    Y_Laplacian_2 = np.zeros((m,n,r))
    
    for dim in range(1,r):
        GRAD = grad(Y[:,:,dim]) 
        GRAD_x = GRAD[:,:,1]
        GRAD_y = GRAD[:,:,2]
        GRAD_norm = np.sqrt(GRAD_x**2 + GRAD_y**2)
        GRAD_norm_thresh = wthresh(GRAD_norm, 'h', h)#странная функция wthresh, нужно найти                 
        ind = np.where(GRAD_norm_thresh == 0)
        GRAD_x[ind] = 0
        GRAD_y[ind] = 0
        GRAD_thresh[:,:,1] = GRAD_x
        GRAD_thresh[:,:,2] = GRAD_y                                       
        Y_Laplacian_2[:,:,dim] = div(grad(div( GRAD_thresh)))        
        
    rhs = Y_Laplacian_2 + epsilon * Y    
        
    for dim in range(1,r):
        T[:,:,dim] = PoissonDCT_variant(rhs[:,:,dim], 1, 0, epsilon)
    return T




# solve the equation  (mu*L^2 - lambda*L + epsilon)*u = rhs via DCT
# where L means Laplacian operator 
def PoissonDCT_variant(rhs, mu, lambda_, epsilon) : 

  M,N=rhs.shape

  k=np.arange(1,M)
  l=np.arange(1,N)
  eN=np.ones((1,N))
  eM=np.ones((M,1))
  k=np.cos(np.pi/M*(k-1))
  l=np.cos(np.pi/N*(l-1));
  k=kron(k,eN)#странная функция kron, надо найти
  l=kron(eM,l)#аналогично предыдущему

  kappa=2*(k+l-2);
  const = mu * kappa**2 - lambda_ * kappa + epsilon;
  u=dct2(rhs)#странная функция dct2, надо найти
  u=u/const
  u=idct2(u)#аналогично предыдущему обратная функция
  return u



#% compute the gradient of a 2D image array

def grad(A):

  m,n=A.shape
  B=np.zeros((m,n,2))

  Ar=np.zeros((m,n))
  Ar[:,1:n-1]=A[:,2:n]
  Ar[:,n]=A[:,n]


  Au=np.zeros((m,n))
  Au[1:m-1,:]=A[2:m,:]
  Au[m,:]=A[m,:]

  B[:,:,1]=Ar-A     
  B[:,:,2]=Au-A
  return B     


# % compute the divergence of gradient
# % Input A is a matrix of size m*n*2
# % A(:,:,1) is the derivative along the x direction
# % A(:,:,2) is the derivative along the y direction

def div(A) : 

  m,n,_=A.shape
  B=np.zeros((m,n))

  T=A[:,:,1]
  T1=np.zeros((m,n))
  T1[:,2:n]=T[:,1:n-1]

  B=B+T-T1

  T=A[:,:,2]
  T1=zeros((m,n))
  T1[2:m,:]=T[1:m-1,:]

  B=B+T-T1
  return B